# Data Mining Course Spark Exercise
## Sharif University of Technology

In this notebook we are going to analyze farsi wikipedia. 
Outline of the exercise:
* Dataset preparation (5 Points)
* Preprocessing (20 Points) 
* Exploration (15 Points) 
* TF-IDF + Search (30 Points)
* K-Means Clustering (30 Points)

You should replace the `TODO` parts with your implementation. Remeber that each `TODO` may take multiple lines and you shouldn't limit your self to one-line codes.

## Prerequisites
You should be faimilar with [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). In this notebook you should use the following formula for tf-idf:
$$f_{t,d}/len(d) \times log(1 + \frac{N}{n_t})$$

You will be implementing distribuited K-means. You can familiarize your self using this [article](https://stanford.edu/~rezab/classes/cme323/S16/projects_reports/bodoia.pdf)

## Warning: RDD api only
You **can not** use Dataframe, Dataset, mllib, ml, ... apis of spark in this exercise. You should only use the [RDD api](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD).

# Section 0: Please enter your name below
# Name: رضا رمضان پور
# Student Number: 95101558

# Section 1: Dataset preparation (5 Points)

This section of notebook contains only shell commands. You don't need to completely understand each command or change anything.

Please run all the paragraphs sequentially.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=a65c7a3042ea0b210a6fb4cec3205ad26c4923548ec3762ea4a3a71f069e8bc8
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-11-03 08:37:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.187.253, 3.230.235.205, 35.153.20.238, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.187.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.61MB/s    in 2.0s    

2020-11-03 08:37:30 (6.61 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://85873bfb06a9.ngrok.io


In [6]:
%%shell

sudo apt-get -y install wget git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


## Download the dump

In [7]:
%%shell

wget -P . https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2

--2020-11-03 08:38:01--  https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.your.org (dumps.wikimedia.your.org)... 204.9.55.82, 2001:4978:1:420::cc09:3752
Connecting to dumps.wikimedia.your.org (dumps.wikimedia.your.org)|204.9.55.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883400343 (842M) [application/x-bzip]
Saving to: ‘./fawiki-latest-pages-articles-multistream.xml.bz2’

fawiki-latest-pages 100%[===================>] 842.48M  16.6MB/s    in 61s     

2020-11-03 08:39:03 (13.9 MB/s) - ‘./fawiki-latest-pages-articles-multistream.xml.bz2’ saved [883400343/883400343]



## Extract the dump (this may take a few minutes)

In [8]:
%%shell

bzip2 -d fawiki-latest-pages-articles-multistream.xml.bz2

## Clone git project for converting wikipedia xml dump to json

In [9]:
%%shell

pip install wikiextractor

  Created wheel for wikiextractor: filename=wikiextractor-0.1-cp36-none-any.whl size=38821 sha256=2a2a92c25bedf2cdbda06aad5c4602e33c73a7c5162ff67695df63e19d980209
  Stored in directory: /root/.cache/pip/wheels/10/59/30/8e5a627a070165141e54816359f344497523d6273e0ced6aec
Successfully built wikiextractor


## Run the script to convert xml to json (this might take around 30 minutes)

In [10]:
%%shell

python -m wikiextractor.WikiExtractor -q --json fawiki-latest-pages-articles-multistream.xml

# Ensure output files exist

In [11]:
%%shell

ls text/*/*

text/AA/wiki_00  text/AC/wiki_31  text/AE/wiki_62  text/AG/wiki_93
text/AA/wiki_01  text/AC/wiki_32  text/AE/wiki_63  text/AG/wiki_94
text/AA/wiki_02  text/AC/wiki_33  text/AE/wiki_64  text/AG/wiki_95
text/AA/wiki_03  text/AC/wiki_34  text/AE/wiki_65  text/AG/wiki_96
text/AA/wiki_04  text/AC/wiki_35  text/AE/wiki_66  text/AG/wiki_97
text/AA/wiki_05  text/AC/wiki_36  text/AE/wiki_67  text/AG/wiki_98
text/AA/wiki_06  text/AC/wiki_37  text/AE/wiki_68  text/AG/wiki_99
text/AA/wiki_07  text/AC/wiki_38  text/AE/wiki_69  text/AH/wiki_00
text/AA/wiki_08  text/AC/wiki_39  text/AE/wiki_70  text/AH/wiki_01
text/AA/wiki_09  text/AC/wiki_40  text/AE/wiki_71  text/AH/wiki_02
text/AA/wiki_10  text/AC/wiki_41  text/AE/wiki_72  text/AH/wiki_03
text/AA/wiki_11  text/AC/wiki_42  text/AE/wiki_73  text/AH/wiki_04
text/AA/wiki_12  text/AC/wiki_43  text/AE/wiki_74  text/AH/wiki_05
text/AA/wiki_13  text/AC/wiki_44  text/AE/wiki_75  text/AH/wiki_06
text/AA/wiki_14  text/AC/wiki_45  text/AE/wiki_76  text/AH/wik

## Reading the data

In [12]:
files = ['text/AA/*']
articles_rdd = sc.textFile(','.join(files)) # Now you have a RDD with wikipedia posts
articles_rdd.take(1)

['{"id": "12125", "url": "https://fa.wikipedia.org/wiki?curid=12125", "title": "علی منصور", "text": "علی منصور\\n\\nرجبعلی منصور (۱۲۶۵ – ۲۱ آذر ۱۳۵۳ تهران)، ملقب به منصورالملک، سیاست\u200cمدار دوره قاجار و پهلوی و نخست\u200cوزیر ایران در دوران رضا شاه و محمدرضا شاه پهلوی بود.\\n\\nپدرش میرزا علی\u200cخان آشتیانی بود. تحصیلات مقدماتی و متوسطه را در مدارس مظفری و دارالفنون به پایان رساند و مطابق معمول زمان در حسن خط و شیوه نگارش نیز جد و جهدی از خود بروز داد. سپس وارد مدرسه سیاسی شد و در ۱۲۸۵ از این مدرسه فارغ\u200cالتحصیل شد و به استخدام وزارت امور خارجه درآمد. \\n\\nاستخدامش در وزارت خارجه مقارن بود با اوائل جنبش مشروطه و سال\u200cها در اداره انگلیس آن وزارتخانه خدمت کرد که ریاستش با حسن رئیس ظهیرالملک بود. با دختر رئیس خود ازدواج کرد و در سال ۱۲۹۴ به ریاست اداره انگلیس رسید. در همین سمت لقب منصورالملک گرفت و علاوه بر سمت دولتی خود، در مدرسه سیاسی به تدریس حقوق در یونان و روم قدیم پرداخت. گفته شده که منصورالملک در سمت رئیس اداره انگلیس در انعقاد قرارداد ۱۹۱۹ نقش مهمی داشت.\\n\\nعلی منص

# Section 2: Preprocessing (20 Points)

In this section we will find and remove stop words and also remove the words with a low count in the corpus

In [13]:
import json
import re 
perisan_alphabet = "آ-ی"
#zero_width_none_joiner
zwnj = "\u200C"
persian_num = "\u0660-\u0669\u06F0-\u06F9"
vowels = "\u064E\u0650\u064F\u064B\u064D\u064C"
hamze  = "\u0621"
tashdid = "\u0651"
word_reg = "[" + perisan_alphabet + zwnj + persian_num + vowels + hamze + tashdid+ "]" + "+"

def word_regex(string):
  return re.findall(word_reg, string)
  
def word_extractor(wiki):
  words = []
  words.extend(word_regex(wiki['title']))
  words.extend(word_regex(wiki['text']))
  tuples = [(word, wiki['id']) for word in words]
  return tuples

def article_remove_sp(article):
  for word in stopwords:
        #start of line
        article['title']= re.sub('^'+word+'\s+','', article['title'])
        article['text']= re.sub('^'+word+'\s+','', article['text'])
        
        #middle of line
        article['title']= re.sub(r"(\s+|\()"+word+ "(\s+|\))", r'\1\2', article['title'])
        article['text']= re.sub(r"(\s+|\()"+word+ "(\s+|\))", r'\1\2', article['text'])
        
        #end of line
        article['title']= re.sub(r'\s+'+word+"([$.،:;]+)",r'\1', article['title'])
        article['text']= re.sub(r'\s+'+word+"([$.،:;]+)",r'\1', article['text'])
        
  return article


  
articles_rdd = articles_rdd.map(json.loads) # parse the json string
articles_rdd.take(1)



[{'id': '12125',
  'text': 'علی منصور\n\nرجبعلی منصور (۱۲۶۵ – ۲۱ آذر ۱۳۵۳ تهران)، ملقب به منصورالملک، سیاست\u200cمدار دوره قاجار و پهلوی و نخست\u200cوزیر ایران در دوران رضا شاه و محمدرضا شاه پهلوی بود.\n\nپدرش میرزا علی\u200cخان آشتیانی بود. تحصیلات مقدماتی و متوسطه را در مدارس مظفری و دارالفنون به پایان رساند و مطابق معمول زمان در حسن خط و شیوه نگارش نیز جد و جهدی از خود بروز داد. سپس وارد مدرسه سیاسی شد و در ۱۲۸۵ از این مدرسه فارغ\u200cالتحصیل شد و به استخدام وزارت امور خارجه درآمد. \n\nاستخدامش در وزارت خارجه مقارن بود با اوائل جنبش مشروطه و سال\u200cها در اداره انگلیس آن وزارتخانه خدمت کرد که ریاستش با حسن رئیس ظهیرالملک بود. با دختر رئیس خود ازدواج کرد و در سال ۱۲۹۴ به ریاست اداره انگلیس رسید. در همین سمت لقب منصورالملک گرفت و علاوه بر سمت دولتی خود، در مدرسه سیاسی به تدریس حقوق در یونان و روم قدیم پرداخت. گفته شده که منصورالملک در سمت رئیس اداره انگلیس در انعقاد قرارداد ۱۹۱۹ نقش مهمی داشت.\n\nعلی منصور در سال ۱۲۹۸ به کفالت وزارت خارجه منصوب شد. پس از سقوط وثوق\u200cالدوله در سمت 

In [14]:
words_rdd = articles_rdd.flatMap(word_extractor)# extract words from title and description
words_rdd.take(1)


[('علی', '12125')]

In [15]:
words_count_rdd = words_rdd.map(lambda word : (word[0], ([word[1]], 1))).reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1])) # make an rdd with the count of each word
print(words_count_rdd.take(1))


[('رساند', (['12125', '12125', '12125', '12152', '12227', '12234', '12566', '12598', '12608', '12608', '12650', '12662', '9051', '9073', '9084', '9121', '9396', '9400', '9404', '8094', '8094', '8094', '8094', '8105', '8681', '8681', '8681', '8681', '8681', '8681', '8681', '2494', '2494', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2608', '2631', '2645', '2645', '35858', '35858', '35858', '35858', '35963', '35963', '36039', '36237', '36325', '36424', '36823', '36823', '10437', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10502', '10581', '10608', '10608', '10656', '10656', '10656', '10665', '10678', '10678', '10678', '10678', '

In [16]:
top_100 = words_count_rdd.sortBy(lambda x : x[1][1], ascending = False).take(100)# find the 100 most common words
#print(top_100[0])
stopwords = ['در', 'با', 'و', 'شده', 'دارد', 'او', 'برای', 'بود', 'شد', 'آن', 'یک', 'را', 'این', 'است', 'که', 'از', 'به', 'و', 'کرد', 'پس', 'یکی', 'عنوان', 'دو', 'وی', 'نیز', 'واقع', 'یا', 'تا', 'شده', 'خود', 'بر', 'بوده‌شود', 'هر', 'اما', 'همچنین', 'هم', 'دیگر', 'بخش', 'توسط', 'می‌کند', 'آن‌است', 'می‌باشد', 'هستند', 'مورد', 'باشد', 'وجود', 'داشته', 'دارای', 'انجام', 'می', 'بعد', 'می‌شود', 'بین', 'داد', 'شود', 'زمان', 'زیر', 'داشت', 'روی', 'بودند', 'کنند']

In [17]:
articles_without_stopwords_rdd = articles_rdd.map(article_remove_sp) # remove stopwords from the article title and text
articles_without_stopwords_rdd.take(1)

[{'id': '12125',
  'text': 'علی منصور\n\nرجبعلی منصور (۱۲۶۵ – ۲۱ آذر ۱۳۵۳ تهران)، ملقب  منصورالملک، سیاست\u200cمدار دوره قاجار  پهلوی  نخست\u200cوزیر ایران  دوران رضا شاه  محمدرضا شاه پهلوی.\n\nپدرش میرزا علی\u200cخان آشتیانی. تحصیلات مقدماتی  متوسطه   مدارس مظفری  دارالفنون  پایان رساند  مطابق معمول   حسن خط  شیوه نگارش  جد  جهدی   بروز. سپس وارد مدرسه سیاسی    ۱۲۸۵   مدرسه فارغ\u200cالتحصیل    استخدام وزارت امور خارجه درآمد. \n\nاستخدامش  وزارت خارجه مقارن   اوائل جنبش مشروطه  سال\u200cها  اداره انگلیس  وزارتخانه خدمت   ریاستش  حسن رئیس ظهیرالملک.  دختر رئیس  ازدواج    سال ۱۲۹۴  ریاست اداره انگلیس رسید.  همین سمت لقب منصورالملک گرفت  علاوه  سمت دولتی،  مدرسه سیاسی  تدریس حقوق  یونان  روم قدیم پرداخت. گفته   منصورالملک  سمت رئیس اداره انگلیس  انعقاد قرارداد ۱۹۱۹ نقش مهمی.\n\nعلی منصور  سال ۱۲۹۸  کفالت وزارت خارجه منصوب.   سقوط وثوق\u200cالدوله  سمت معاون میرزا اسدالله خان مشارالسلطنه، وزیر امور خارجه ابقا.  کابینه\u200cهای سپهدار رشتی  سیدضیاءالدین همچنان معاون وزیر امور خارجه.  کابین

In [18]:
import numpy as np
def word_mapper(word):
  tuples = []
  for id in list(word[1]):
    tuples.extend((id, word[0]))
    
  return tuples

def id_extractor(word):
  id_list = np.unique(word[1][0])
  out_list = []
  for id in id_list:
    out_list.append((id, [word[0]]))
    
  return out_list

MIN_COUNT = 20
uncommon_words = words_count_rdd.filter(lambda x : x[1][1] < MIN_COUNT).map(lambda x : x[0]).collect() # list of the words that have occured less than MIN_COUNT in the whole corpus
id_uncommon = words_count_rdd.filter(lambda x : x[1][1] < MIN_COUNT).flatMap(id_extractor).reduceByKey(lambda x,y: x+y)
#joined_wrods_id = words_id_rdd.filter(lambda x : x[0] in uncommon_words).flatMap(word_mapper).groupByKey()
print("10 Sample uncommon words :")
#print(uncommon_words[0:10])
#print("Length of uncommon words = " + str(len(uncommon_words)))

10 Sample uncommon words :


In [19]:
def article_remove_uncommon(article):
  wiki = article[1][0]
  word_list = article[1][1]
  for word in word_list:
        wiki['title']= re.sub(word,'', wiki['title'])
        wiki['text']= re.sub(word,'', wiki['text'])
        #start of line
        #article['title']= re.sub('^'+word+'\s+','', article['title'])
        #article['text']= re.sub('^'+word+'\s+','', article['text'])


        #middle of line
        #article['title']= re.sub(r"(\s+|\()"+word+ "(\s+|\))", r'\1\2', article['title'])
        #article['text']= re.sub(r"(\s+|\()"+word+ "(\s+|\))", r'\1\2', article['text'])

        #end of line
        #article['title']= re.sub(r'\s+'+word+"([$.،:;]+)",r'\1', article['title'])
        #article['text']= re.sub(r'\s+'+word+"([$.،:;]+)",r'\1', article['text'])
        
  return wiki
    

articles_cleaned_rdd = articles_without_stopwords_rdd.map(lambda wiki: (wiki['id'], wiki)).join(id_uncommon).map(article_remove_uncommon) #remove uncommon words from articles_cleaned_rdd
#articles_cleaned_rdd.take(1)

# Section 3: Exploration (15 Points)
Please answer the following questions regarding the dataset:
* How many unique words remain after the cleaning procedure?


In [21]:
#number of word
remove_words = stopwords + uncommon_words
num_word = words_count_rdd.filter(lambda x : x[0] not in remove_words).collect()

print("Number of unique words : " + str(len(num_word)))

Number of unique words : 24699


# Section 4: TF-IDF + Search (30 Points)

In [22]:
import math
from operator import add
# number of articles
num_docs = articles_rdd.count()

def vector_tf(wiki):
  words = []
  words.extend(word_regex(wiki['title']))
  words.extend(word_regex(wiki['text']))
  num_word_wiki = len(words)
  
  out_tuples = []
  for word in num_word:
    word_count = words.count(word[0])
    if (num_word_wiki is not 0) and (word_count is not 0):
      out_tuples.append(((word[0], wiki['id']), words.count(word[0])/num_word_wiki))
      
  return out_tuples

def add_tfidf(wiki):
  article = wiki[1][0]
  article['vector'] = wiki[1][1]
  return article

def list_to_dict(wiki):
  out_dict = dict(list(wiki[1]))
  return (wiki[0], out_dict)

# Length of tf-idf vector is the same as num_word (num_word is words after cleaning)
# using flatMap to find tf vector of each article and return it as ((word, article_id), tf) ,then i map it to (word, (aticle_id, tf))
tf_rdd = articles_cleaned_rdd.flatMap(vector_tf).map(lambda tuples: (tuples[0][0], (tuples[0][1], tuples[1])))
# now i use tf_rdd to find document freq, we now if tf of one word in one article is greater than zero, then it is in that article
word_df_rdd = tf_rdd.filter(lambda tuples: tuples[1][1] > 0 ).map(lambda tuples: (tuples[0], 1)).reduceByKey(lambda x,y: x+y).map(lambda tuples : (tuples[0], tuples[1]/num_docs)) # calculate document frequncy for each word
# now by word_df we calculate idf vector (document frequency might be zero so i add 1 to it so : idf = 1 + N / (n_t + 1))
idf_rdd = word_df_rdd.map(lambda tuples : (tuples[0], math.log10(1+ 1/(tuples[1]+1))))
# then i join tf and idf, and then multiply them
tf_idf_rdd = tf_rdd.join(idf_rdd).map(lambda tuples: (tuples[1][0][0], (tuples[0], tuples[1][0][1]*tuples[1][1])))
tf_grouped_dict = tf_idf_rdd.groupByKey().map(list_to_dict)
# adding tf_idf to articles
articles_tf_idf_vectors = articles_cleaned_rdd.map(lambda wiki: (wiki['id'], wiki)).join(tf_grouped_dict).map(add_tfidf)#TODO: add `vector` key to articles_cleaned_rdd dictionary with the tf_idf dictionary
articles_tf_idf_vectors.take(1)
#ativ_sample = atiarticles_tf_idf_vectors.take(1)


[{'id': '11760',
  'text': 'سال کبیسه\n\nواژهٔ عربی کبیسه  ریشهٔ «» (فعل  ) به\u200cمعنی پُر کردن چاه  خاک است؛   گاه\u200cشماری، کبیسه عبارت   تعداد روز  روزهایی   رفع  سال\u200cهای عمومی (تأخیر به\u200cدلیل   چند سال)  مقدار واقعی   رفع نقصان سال\u200cهای خاص نسبت  سال\u200cهای،  سال معینی می\u200cافزایند. سال کبیسه سالی   مقدار نقصان   افزوده.\n\n  خورشیدی تقریباً  چهار سال    قمری تقریباً  سه سال  روز کبیسه.   خورشیدی قمری، مثل گاه\u200cشماری عبری  گاه\u200cشماری چینی، تقریباً  سه سال  ماه کبیسه.\n\n  مبتنی  تقویم خورشیدی، مثل تقویم گریگوری  تقویم هجری خورشیدی  سال\u200cهای عادی   سال\u200cهای کبیسه آن\u200cها ، به\u200cطور متوسط  حدود   سال، هشت سال کبیسه   سال عادی.   معمولاً   چهار سال  سال کبیسه   علاوه    تعدیل دقیق\u200cتر  دوره\u200cهایی بزرگتر،  حذف  افزودن فاصله  بعضی ،  آن\u200cها کاسته.\n\nی تقویم هجری شمسی   کشور  طریق محاسبه لحظه تحویل سال  مقایسه   لحظه ظهر حقیقی  نصف النهار رسمی  کشور تعیین.  اینجا    حالت  ممکن  اتفاق بیفتد:\n تقویم هجری خورشیدی علاوه  کبیسه چهارسال

## Searching

In [23]:
from scipy.spatial.distance import cosine
# i am not storing anything in dataframe
import pandas as pd

import numpy as np

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def cosine_com(a, b):
    key_a = a.keys()
    key_b = b.keys()
    key_i = intersection(key_a, key_b)
    val_a = [a[x] for x in key_i]
    val_b = [b[x] for x in key_i]
    dot_all = np.dot(val_a, val_b)
    norm_a = np.linalg.norm(list(a.values()))
    norm_b = np.linalg.norm(list(b.values()))
    return dot_all / (norm_a * norm_b)

def query_word_find(wiki):
  out_tuple = []
  for word in query_words:
    if(  word in wiki['title'] or word in wiki['text']):
      out_tuple.append((word, 1))
  return  out_tuple  

query = 'هخامنشیان ساسانیان هگمتانه'
query_words = query.split(' ')
query_count = len(query_words)
tf_query = {word : (query.count(word))/query_count  for word in query_words}
wd_list = articles_cleaned_rdd.flatMap(query_word_find).reduceByKey(lambda x,y : x+y).collect()
wd_dict = dict([(tuples[0], tuples[1]/num_docs) for tuples in wd_list])
tf_idf_query = {word: tf * wd_dict[word] for word, tf in tf_query.items()}
top_10_similar = articles_tf_idf_vectors.sortBy(lambda wiki: cosine_com(wiki['vector'], tf_idf_query), ascending= False).take(10)
for article in top_10_similar:
    print(article['title'])
    print(article['url'])
    

شاهنشاهی ساسانی
https://fa.wikipedia.org/wiki?curid=2670
۲۲۵ (میلادی)
https://fa.wikipedia.org/wiki?curid=5319

https://fa.wikipedia.org/wiki?curid=35393
دجله
https://fa.wikipedia.org/wiki?curid=8231
سروان
https://fa.wikipedia.org/wiki?curid=1939
خانه
https://fa.wikipedia.org/wiki?curid=18977

https://fa.wikipedia.org/wiki?curid=16176
تیسفون
https://fa.wikipedia.org/wiki?curid=13186
تخت سلیمان
https://fa.wikipedia.org/wiki?curid=1672
چیرگی اشکانیان  ارمنستان
https://fa.wikipedia.org/wiki?curid=1827


# Section 5: K-Means clustering (30 Points)

### Random initialization

In [24]:
import random
CENTROID_NUM = 100

centroids = []
for i in range(CENTROID_NUM):
  random_vector = dict([(num_word[i][0], random.uniform(0, 1)) for i in range(len(num_word))])# Initialize a random array with the size of tf-idf vectors
  centroids.append(random_vector)

### Update centroids

In [25]:
from collections import Counter

def centroid_assigner(article):
  cosine_sim = []
  for i in range(CENTROID_NUM):
      cosine_sim.append(cosine_com(article['vector'], centroids[i]))
    
  argmax_centroid = np.argmax(cosine_sim)
  return (argmax_centroid, article)
  
def new_centroid_f(list_article):
  articles = list(list_article[1])
  len_articles = len(articles)
  sum_articles = {word[0]:0 for word in num_word}
  for i in range(len_articles):
    for key in sum_articles: 
      if key in articles[i][1]['vector']: 
          sum_articles[key] = sum_articles[key] + articles[i][1]['vector'][key]
  for key in sum_articles:
    sum_articles[key] = sum_articles[key] / len_articles
    
  new_centroid =  sum_articles
  return (new_centroid , articles[0][0])

def find_distance():
  dist_sum = 0
  for i in range(CENTROID_NUM):
    dist_sum += (1 - cosine_com(updated_centroids[i], centroids[i]))

  return dist_sum
EPOCHS = 3


for i in range(EPOCHS):
  vector_centroid_assignments_rdd = articles_tf_idf_vectors.map(centroid_assigner).map(lambda x : (x[0], (i, x[1]))) #assign the nearest centroid to each article
  updated_centroids = vector_centroid_assignments_rdd.groupByKey().map(new_centroid_f).sortBy(lambda x: x[1]).map(lambda x: x[0]).collect() #update centroids based using a reduce on vector_centroid_assignments_rdd
  distance =  find_distance() # compute the total amount of distance the centroids were moved
  centroids = updated_centroids
  print("Iteration %d" % i)
  print(distance)

Iteration 0
80.54919135112658
Iteration 1
23.04142786496699
Iteration 2
8.05856701135073


In [26]:
def take_10(group):
  list_art = list(group[1])
  
  return_number = 10
  if (len(list_art) < 10):
    return_number = len(list_art)
  return (group[0], list_art[0:return_number])
  
# Collect 10 documents from each cluster and print them
doc_10 = dict(articles_tf_idf_vectors.map(centroid_assigner).groupByKey().sortByKey().map(take_10).collect())
for i in range(CENTROID_NUM):
  print("Two sample Articles from cluster " + str(i) + " :")
  print(doc_10[i][0]['title'])
  print(doc_10[i][0]['url'])
  print(doc_10[i][1]['title'])
  print(doc_10[i][1]['url'])

Two sample Articles from cluster 0 :
شر (خواننده)
https://fa.wikipedia.org/wiki?curid=25887
ناتالی پورتمن
https://fa.wikipedia.org/wiki?curid=32217
Two sample Articles from cluster 1 :
سلاح‌های خودکار
https://fa.wikipedia.org/wiki?curid=23116
ده
https://fa.wikipedia.org/wiki?curid=26563
Two sample Articles from cluster 2 :

https://fa.wikipedia.org/wiki?curid=13283

https://fa.wikipedia.org/wiki?curid=7487
Two sample Articles from cluster 3 :
کتاب کوچه
https://fa.wikipedia.org/wiki?curid=59578
ارسطو
https://fa.wikipedia.org/wiki?curid=2634
Two sample Articles from cluster 4 :
گرشاسپ‌نامه
https://fa.wikipedia.org/wiki?curid=11902
 
https://fa.wikipedia.org/wiki?curid=66058
Two sample Articles from cluster 5 :
 (چشمه)
https://fa.wikipedia.org/wiki?curid=57079
جلفا (آذربایجان شرقی)
https://fa.wikipedia.org/wiki?curid=17101
Two sample Articles from cluster 6 :
جام جم
https://fa.wikipedia.org/wiki?curid=44489
باشگاه فوتبال ابومسلم خراسان
https://fa.wikipedia.org/wiki?curid=40604
Two sample 